In [1]:
few_shot_path = '/scratch/dzhang5/LLM/TWEET-FID/expert.train.short.csv'
data_path = '/scratch/dzhang5/LLM/TWEET-FID/expert.smalltest.csv'
output_dir = '/scratch/dzhang5/LLM/TWEET-FID/test-results-autolabel'
model_name = "meta-llama/Llama-2-7b-chat-hf"
label_column = 'sentence_class'
text_column = 'tweet'
few_shot_num = 8
few_shot_selection = "semantic_similarity"
token_path = "/home/dzhang5/.cache/huggingface/token"
cache=False
console_output=False
temperature=0.1
random_shuffle_examples = True
random_shuffle_examples_seed = 1

In [2]:
from autolabel.schema import ModelProvider, TaskType
from autolabel.models import register_model, MODEL_REGISTRY
from hf_pipeline_new import HFPipelineLLMNew
from few_shot_new import NewAutoLabelConfig, NewExampleSelectorFactory
from autolabel.few_shot import ExampleSelectorFactory
from template_inst import update_inst_mode
from named_entity_recognition_new import NewNamedEntityRecognitionTask
from classification_new import NewClassificationTask
from autolabel.tasks import TASK_TYPE_TO_IMPLEMENTATION 
import sys

In [3]:
update_inst_mode(model_name)
TASK_TYPE_TO_IMPLEMENTATION[TaskType.NAMED_ENTITY_RECOGNITION] = NewNamedEntityRecognitionTask
TASK_TYPE_TO_IMPLEMENTATION[TaskType.CLASSIFICATION] = NewClassificationTask
sys.modules['autolabel.labeler'].ExampleSelectorFactory = NewExampleSelectorFactory
register_model(ModelProvider.HUGGINGFACE_PIPELINE, HFPipelineLLMNew)

In [4]:
from autolabel import LabelingAgent, AutolabelDataset
import json
import os
import pandas as pd

In [5]:
with open(token_path) as tfile:
    token_str = tfile.read()

from huggingface_hub import login
login(token=token_str)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/dzhang5/.cache/huggingface/token
Login successful


In [6]:
no_auto = ["microsoft/prophetnet", "microsoft/phi-2", "google/pegasus-x"]
if any([pre in model_name for pre in no_auto]):
    device_map = None
else:
    device_map = "auto"

In [ ]:
refuel_models = ["refuel-llm", "llama-13b-chat"]
if model_name in refuel_models:
    provider = "refuel"
    em_provider = "huggingface_pipeline"
    model_params = {"max_length":4096, "temperature": temperature}
    task_name = f"FoodborneIllnessIncidentTweetClassification_{few_shot_selection}_{model_name}"
elif model_name.startswith('gpt'):
    provider = "openai"
    em_provider = "openai"
    model_params = {"max_tokens":4096, "temperature": temperature}
    task_name = f"FoodborneIllnessIncidentTweetClassification_{few_shot_selection}_{model_name}"
else:
    provider = "huggingface_pipeline"
    em_provider = "huggingface_pipeline"
    model_params = {"max_length":4096, "temperature": temperature,
                    "quantize": 16, "device_map": device_map,
                    "token": token_str}
    task_name = f"FoodborneIllnessIncidentTweetClassification_{few_shot_selection}_{model_name.split('/')[1]}"

In [7]:
if not os.path.exists(output_dir):
    # Create the directory
    os.makedirs(output_dir)
output_name = os.path.split(model_name)[-1] + '_' + few_shot_selection + '_' + os.path.split(data_path)[-1]
output_path = os.path.join(output_dir, output_name)

In [8]:
test_data = pd.read_csv(data_path)

In [9]:
config = {
    "task_name": task_name,
    "task_type": "classification",
    "dataset": {
        "label_column": label_column,
        "text_column": text_column,
        "delimiter": ","
    },
    "model": {
        "provider": provider,
        "name": model_name,
        "params": model_params
    },
    "embedding": {
        "provider": em_provider,
    },
    "prompt": {
        "task_guidelines": ("You are an expert at identifying foodborne illness incident information. For the given text, "
                            "your task is to evaluate the text to determine if it describes a potential foodborne illness event. "
                            '''Please answer with "Yes" if it describes a potential foodborne illness event, otherwise answer with "No".'''
                            "Use the following examples as a guide for your predictions and format your responses similarly."
                  ),
        "output_guidelines": '''You will answer with just the correct output label ("Yes" or "No") and nothing else.''',
        "labels": [
            "Yes",
            "No"
        ],
        "few_shot_examples": few_shot_path,
        "few_shot_selection": few_shot_selection,
        "few_shot_num": few_shot_num,
        "random_shuffle_examples": random_shuffle_examples,
        "random_shuffle_examples_seed": random_shuffle_examples_seed,
        "example_template": f"Input: {{{text_column}}}\nOutput: {{{label_column}}}"
    }
}

config = NewAutoLabelConfig(config)

In [10]:
agent = LabelingAgent(config=config, console_output=console_output, cache=cache)

2024-01-27 00:45:35 accelerate.utils.modeling INFO: We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The model 'LlamaForCausalLM' is not supported for text2text-generation. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'SeamlessM4TForTextToText', 'SeamlessM4Tv2ForTextToText', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'UMT5ForConditionalGeneration', 'XLMProphetNetForConditionalGeneration'].


In [11]:
ds = AutolabelDataset(test_data[[text_column, label_column]], config=config)
agent.plan(ds)

2024-01-27 00:45:48 sentence_transformers.SentenceTransformer INFO: Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2
2024-01-27 00:45:49 sentence_transformers.SentenceTransformer INFO: Use pytorch device: cuda


Batches: 0it [00:00, ?it/s]

Output()

Batches: 0it [00:00, ?it/s]

Batches: 0it [00:00, ?it/s]

...

Batches: 0it [00:00, ?it/s]

┌──────────────────────────┬──────┐
│ Total Estimated Cost     │ $0.0 │
│ Number of Examples       │ 10   │
│ Average cost per example │ $0.0 │
└──────────────────────────┴──────┘

───────────────────────────────────────────────── Prompt Example ──────────────────────────────────────────────────

<s>[INST] <<SYS>>
 You are an expert at identifying foodborne illness incident information. Within the TWEET-FID dataset, the task is
to evaluate a tweet in the TWEET-FID dataset to determine if it describes a potential foodborne illness event. 
Please answer with "Yes" if it describes a potential foodborne illness event, otherwise answer with "No".Use the 
following examples as a guide for your predictions and format your responses similarly.

You will answer with just the correct output label ("Yes" or "No") and nothing else.


Some examples with their output answers are provided below:

Input: @USER As much fun as I can. Woke up with food poisoning or stomach flu. Been bugging me all day #tmi Almost 
done driving for the day
Output: Yes
Input: Today was rough. Think I had a touch of food poisoning earlier. Thought I was gonna die. Yuck. Therabreath 
rinse & mints helped though! lol
Output: Yes
Input: I cant come into work today because I have food poisoning. I went out with the lads last night and we had a 
cheap curry, we all ended up ...
Output: Yes
Input: @USER @USER that's not good, do you know what you ate to get food poisoning?
Output: Yes
Input: I think I got food poison from ABW 
[EMOJI_astonished_face][EMOJI_face_with_medical_mask][EMOJI_face_screaming_in_fear]
Output: Yes
Input: I think I have food poison poops & vomit [EMOJI_persevering_face] that damn subway
Output: Yes
Input: lmao RT @USER I've eaten off the streets in India & Bhutan, nothing, on a cruise ship - food poisoning 
twice! #TNI #YouNeverKnow
Output: Yes
Input: Food poisoning in Mandeville  HTTPURL
Output: No

<</SYS>>


Now I want you to label the following example:

Input: @USER As much fun as I can. Woke up with food poisoning or stomach flu. Been bugging me all day #tmi Almost 
done driving for the day
Output: [/INST]

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [12]:
# now, do the actual labeling
ds = agent.run(ds)

Output()

Batches: 0it [00:00, ?it/s]

Batches: 0it [00:00, ?it/s]

Batches: 0it [00:00, ?it/s]

Batches: 0it [00:00, ?it/s]

...

Batches: 0it [00:00, ?it/s]

/scratch/dzhang5/visa/visa39/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/scratch/dzhang5/visa/visa39/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/scratch/dzhang5/visa/visa39/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


classification_report:
              precision    recall  f1-score   support

          No       0.00      0.00      0.00         2
         Yes       0.80      1.00      0.89         8

    accuracy                           0.80        10
   macro avg       0.40      0.50      0.44        10
weighted avg       0.64      0.80      0.71        10

Actual Cost: 0.0

┏━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ accuracy ┃ support ┃ completion_rate ┃
┡━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ 0.8      │ 10      │ 1.0             │
└──────────┴─────────┴─────────────────┘

In [13]:
metrics = ds.eval()

/scratch/dzhang5/visa/visa39/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/scratch/dzhang5/visa/visa39/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/scratch/dzhang5/visa/visa39/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


┏━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ accuracy ┃ support ┃ completion_rate ┃ classification_report                                 ┃
┡━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 0.8      │ 10      │ 1.0             │               precision    recall  f1-score   support │
│          │         │                 │                                                       │
│          │         │                 │           No       0.00      0.00      0.00         2 │
│          │         │                 │          Yes       0.80      1.00      0.89         8 │
│          │         │                 │                                                       │
│          │         │                 │     accuracy                           0.80        10 │
│          │         │                 │    macro avg       0.40      0.50      0.44        10 │
│          │         │                 │ weighted avg       0.64      0.80      0.71        10 │
│          │         │                 │                                                       │
└──────────┴─────────┴─────────────────┴───────────────────────────────────────────────────────┘

In [17]:
ds.df.to_csv(output_path, index=False)
ds.df.to_pickle(output_path.replace('.csv', '.pkl'))